zip2john2zip
============

## Challenge
Shoot, I forgot the password to our flag archive.. can you try cracking it for me? Here's the hash. Since you don't have the archive that means you can't steal all our flags!

## Included files
hash.txt

In [3]:
%%bash
cat hash.txt

flag.zip/flag.txt:$pkzip2$1*1*2*0*34*28*64ac0ae2*0*26*0*34*64ac*a388*2c386d49756e1d70ab5f2d8b7ccf1703b28d2775e84d89ccf4bf26d0e735e9a817b0032b5071540889c34b9331b694d6042c30a0*$/pkzip2$:flag.txt:flag.zip::flag.zip

Let's try the usual place to start - the rockyou list

In [3]:
%%bash
john hash.txt --wordlist=/usr/share/wordlists/rockyou.txt

Using default input encoding: UTF-8


Loaded 1 password hash (PKZIP [32/64])


Will run 8 OpenMP threads
Press Ctrl-C to abort, or send SIGUSR1 to john process for status


factfinder       (flag.zip/flag.txt)     


1g 0:00:00:00 DONE (2025-11-10 09:19) 2.941g/s 2843Kp/s 2843Kc/s 2843KC/s fmxrider..dolphins_lover
Use the "--show" option to display all of the cracked passwords reliably
Session completed. 


The password was recovered as "factfinder", but that doesn't get us the flag.
Let's look at the encoding of the $pkzip$ file

Based on [https://github.com/openwall/john/issues/4300](https://github.com/openwall/john/issues/4300), the fields are

```
    C Hash count: 1
    B Number of valid bytes in checksum: 1
    DT Data type: 2 (inlined full data)
    MT Magic type: 0 (no file magic used)
    CL Compressed length: 0x34 bytes
    UL Uncompressed length: 0x28 bytes
    CR Final CRC32: 64ac0ae2
    OF Offset: 0
    OX Additional offset: 0x26
    CT Compression type: 0 (stored)
    DL Length of inlined data: 0x34 bytes
    CS Checksum from CRC32: 64ac (this is the MSB of "Final CRC32" above)
    TC Checksum from timestamp: a388
```

So it seems if we decrypt the blob ```2c386d49756e1d70ab5f2d8b7ccf1703b28d2775e84d89ccf4bf26d0e735e9a817b0032b5071540889c34b9331b694d6042c30a0``` we should get back the file.

Let's try to reassemble a zip (with a bit of help from ChatGPT)



In [30]:
import zlib
from pwn import *

with open("hash.txt", "r") as f:
    hash = f.read().split("$pkzip2$")[1].split("*$/pkzip2$")[0]

c, b, dt, mt, cl, ul, cr, of, ox, ct, dl, cs, tc, blob = [int(x,16) for x in hash.split("*")]
blob = unhex(hash.split("*")[-1])
with open("flag.zip", "wb") as f:
    name=b"flag.txt"
    lfh=b"PK\3\4" + struct.pack("<HHHHHIIIHH", 20, 1, ct, 0, 0, cr, cl, ul, len(name), 0) + name # Local file header
    cdfh=b"PK\1\2" + struct.pack("<HHHHHHIIIHHHHHII", 20, 20, 1, ct, 0, 0, cr, cl, ul, len(name), 0, 0, 0, 0, 0, 0) + name
    eocd=b"PK\5\6"+struct.pack("<HHHHIIH",0,0,1,1,len(cdfh),len(lfh)+len(blob),0)
    f.write(lfh)
    f.write(blob)
    f.write(cdfh)
    f.write(eocd)


In [32]:
%%bash
unzip -P "factfinder" flag.zip
cat flag.txt

Archive:  flag.zip
 extracting: flag.txt                
bctf{not_all_hashes_are_hashed_equally}
